<a href="https://colab.research.google.com/github/nataliepham6720/16-745_Optimal_Control/blob/main/hopper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Re-import necessary modules after code execution environment reset
import numpy as np
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf


def kinematics(q):
    return q[2:4]


def visualize(vis, xtraj, dt):
    n_leg = 100
    r_body = 0.2
    r_foot = 0.1
    r_leg = 0.5 * r_foot

    vis["/Background"].set_property("visible", True)
    vis["/Background"].set_property("top_color", 0xFFFFFF)
    vis["/Background"].set_property("bottom_color", 0xFFFFFF)
    vis["/Axes"].set_property("visible", False)

    vis["body"].set_object(g.Sphere(r_body), g.MeshPhongMaterial(color=0x00FF00))
    vis["foot"].set_object(g.Sphere(r_foot), g.MeshPhongMaterial(color=0xFFA500))

    for i in range(n_leg):
        vis[f"leg{i}"].set_object(g.Sphere(r_leg), g.MeshPhongMaterial(color=0x000000))

    anim = vis.animation()
    for t in range(xtraj.shape[1]):
        q = xtraj[:, t]
        p_body = np.array([q[0], 0.0, q[1]])
        p_foot = np.array([q[2], 0.0, q[3]])

        dir_vec = np.array([q[2] - q[0], q[3] - q[1]])
        norm_dir = np.linalg.norm(dir_vec)
        if norm_dir == 0:
            dir_unit = np.zeros(2)
        else:
            dir_unit = dir_vec / norm_dir
        r_range = np.linspace(0, norm_dir, n_leg)

        p_leg = [np.array([q[0] + s * dir_unit[0], 0.0, q[1] + s * dir_unit[1]]) for s in r_range]

        z_shift = np.array([0.0, 0.0, r_foot])

        with anim.at_frame(t):
            vis["body"].set_transform(tf.translation_matrix(p_body + z_shift))
            vis["foot"].set_transform(tf.translation_matrix(p_foot + z_shift))
            for i in range(n_leg):
                vis[f"leg{i}"].set_transform(tf.translation_matrix(p_leg[i] + z_shift))

    vis.set_animation(anim)


# Create visualizer
vis = meshcat.Visualizer().open()
